In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import mannwhitneyu
import itertools

import matplotlib.pyplot as plt

import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("data/move.csv", index_col=0)
df.head(3)

,metro,price,way,views,provider,fee_percent,storey,minutes,storeys,living_area,kitchen_area,total_area
0,Planernaia,45000,walk,513,realtor,50,7,10,12,19,8,38
1,VDNKh,50000,walk,389,realtor,50,16,10,16,18,8,41
2,Alekseevskaia,50000,walk,483,realtor,50,5,3,12,19,5,33


In [5]:
sorted(df['way'].unique())

['transport', 'walk']

In [59]:
num_cols = ['minutes', 'living_area', 'kitchen_area', 'total_area']
cat_cols = ['metro', 'way']
target = 'price'

In [60]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    

class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]
    
class NumericPower(BaseEstimator, TransformerMixin):
    def __init__(self, key, p = 2):
        self.key = key
        self.columns = []
        self.p = p+1

    def fit(self, X, y=None):
        B = [self.key + str(i) for i in range(1, self.p)]
        self.columns = B + ['log']
        return self

    def transform(self, X):
        Xp = X.values.reshape(-1,1)
        for i in range(2, self.p):
            Xp = np.hstack([Xp,(X.values.reshape(-1,1) ** i).astype(float)])

        Xp = np.hstack([Xp, np.log(X.values.reshape(-1,1) + 1).astype(float)])    
        B = pd.DataFrame(data = Xp, index = X.index,columns =[self.columns] )
        return B[self.columns]     

In [61]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['price', 'views', 'provider', 'fee_percent', 'storey', 'storeys'], axis=1), df['price'], test_size=0.3, random_state=2)
X_train.shape, X_test.shape

((1012, 6), (434, 6))

In [62]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1012 entries, 336 to 1192
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   metro         1012 non-null   object
 1   way           1012 non-null   object
 2   minutes       1012 non-null   int64 
 3   living_area   1012 non-null   int64 
 4   kitchen_area  1012 non-null   int64 
 5   total_area    1012 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 55.3+ KB


In [63]:
final_transformers = list()

for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    final_transformers.append((cat_col, cat_transformer))


for num_col in num_cols:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=num_col)),
                ('pow_2', NumericPower(key=num_col, p=3)),
                ('Scale', StandardScaler())
            ])
    final_transformers.append((num_col, cont_transformer))

In [68]:
from sklearn.pipeline import FeatureUnion
feats = FeatureUnion(final_transformers)

pipeline = Pipeline([
    ('features', feats),
    ('classifier', GradientBoostingRegressor(n_estimators=500, max_depth=4, random_state=42)),
])

In [69]:
pipeline.fit(X_train, y_train)
preds = pipeline.predict(X_test)
mean_squared_error(y_test, preds)

64209555.671964176

In [70]:
mean_absolute_error(y_test, preds)

2216.83811264012

In [71]:
r2_score(y_test, preds)

0.9120667159802802

In [1]:
import pickle

In [3]:
pickle.load(open('model/model.pkl', 'rb'))

AttributeError: Can't get attribute 'FeatureSelector' on <module '__main__'>